In [1]:
import pandas as pd 
import numpy as np
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('ratings_small.csv',sep = ',',names = ['userID','movieID','rating','timestamp'])
df = df.drop(columns = ['timestamp'])
df.head()

,userID,movieID,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [3]:
df.shape

(100004, 3)

In [4]:
df.describe()

,userID,movieID,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


In [3]:
df["rating"] = df["rating"].convert_objects(convert_numeric=True)
movie_ratings = pd.DataFrame(df.groupby('movieID')['rating'].mean())
movie_ratings.head()

,rating
movieID,
1,3.872470
2,3.401869
3,3.161017
4,2.384615
5,3.267857


In [4]:
mu = df.rating.mean()
mu

3.543608255669773

In [5]:
user_ratings = pd.DataFrame(df.groupby('userID')['rating'].mean())
user_ratings.head()

,rating
userID,
1,2.550000
2,3.486842
3,3.568627
4,4.348039
5,3.910000


In [6]:
movie_matrix = df.pivot_table(index='userID', columns='movieID', values='rating')
movie_matrix.head()

movieID,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userID,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
movie_matrix.shape

(671, 9066)

In [7]:
movie_ratings = movie_ratings.sort_values('rating', ascending=False)
movie_ratings.head()

,rating
movieID,
163949,5.0
71180,5.0
51471,5.0
52617,5.0
53887,5.0


In [8]:
user_ratings = user_ratings.sort_values('rating', ascending=False)
user_ratings.head()

,rating
userID,
46,4.948718
443,4.850000
298,4.800000
448,4.750000
622,4.725806


In [10]:
user_ratings.loc[46]

rating    4.948718
Name: 46, dtype: float64

In [20]:
def rating_calculate(uid,movid):
    movie_vector = movie_matrix[movid]
    movie_matrix_T = movie_matrix.T
    movie_user_matrix_T = movie_matrix_T[movie_matrix_T[uid].notnull()]
    movie_user_matrix = movie_user_matrix_T.T
    movie_similarity = movie_user_matrix.corrwith(movie_vector,method='pearson')
    similar_movie = pd.DataFrame(movie_similarity, columns=['similarity'])
    similar_movie.dropna(inplace=True)
    similar_movie = similar_movie.sort_values(by='similarity',ascending = False)
    similar_movie = similar_movie.reset_index()
    count = 0
    num = 0
    den = 0
    user_base = user_ratings.loc[uid]
    movie_base = movie_ratings.loc[movid]
    for rec in similar_movie.itertuples():
        if rec[1] != movid:
            if count <= 25:
                sim = float(rec[2])
                mid = rec[1]
                item_base = movie_ratings.loc[mid]
                prd = (movie_matrix[mid][uid])-mu-(item_base-mu)-(user_base-mu)
                if not math.isnan(prd):
                    num = num + (sim*prd)
                    den = den+sim
                    rat = num/den
                    count = count+1
    rat = rat+mu+(user_base-mu)+(movie_base-mu)
    return rat

In [21]:
rating = rating_calculate(100,10)
rating

rating    3.277517
dtype: float64